# Lab 2: `keras` and Neural Networks

## Date:  Saturday, October 20th 2018

#### Authors: David Sondak and Pavlos Protopapas

In lab 1, we created our own neural network by writing some simple `python` functions.  We focused on a regression problem wherein we tried to learn a function.  We practiced using the logistic activation function in a network with multiple nodes, but a single hidden layer.  Some of the key observations were:
* Increasing the number of nodes allows us to represent more complicated functions  
* The weights and biases have a very big impact on the solution
* Finding the "correct" weights and biases is really hard to do manually
* There must be a better method for determining the weights and biases automatically

We also didn't assess the effects of different activation functions or different network depths.

Today, we will repeat some of the analysis from last time, but in the lab we will use [`keras`](https://keras.io/).

## I. Architectures Encoding Functions

**Motivating Question:** What exact kind of mathematical/statistical object is encoded by a neural network?

For a network with the following architecture (a single output and one hidden layer), write the closed form expression for the function $f$ represented by the network.

Assume that the activation function at the output node is the identity funciton. Use the following notation:
- let $\mathbf{x}\in \mathbb{R}^D$ be the input; let the components of $\mathbf{x}$ be indexed by $d$
- let $H$ be the total number of hidden nodes, indexed by $h$
- let $\phi_h$ be the activation function at the hidden node $h$
- let $\mathbf{u}_h \in \mathbb{R}^D$ be the weights connecting the input to the $h$-th hidden node
- let $\mathbf{a} \in \mathbb{R}^H$ be the bias for the hidden layer
- let $\mathbf{v} \in \mathbb{R}^H$ be the weights connecting the hidden nodes to the output
- let $b \in \mathbb{R}$ be the bias for the output layer

![](figs/single_hidden_layer.jpg)

For each hidden node $h$, a linear combination of the input, $\mathbf{u}_h^\top \mathbf{x} + \mathbf{a}$, is transformed by the activation function $\phi_h$. Thus, the output of each hidden node $h$ is
$$
\phi_h(\mathbf{u}_h^\top \mathbf{x} + \mathbf{a}).
$$
At the output node, a linear combination of the hidden nodes is taken. Since the activation function here is the identity, the final output of the MLP is
$$
\begin{aligned}
f(\mathbf{x}) = b + \sum_{h=1}^H v_h\phi_h(\mathbf{u}_h^\top \mathbf{x} + \mathbf{a}).
\end{aligned}
$$

## II. Regression

**Motivating Question:** We saw in Part I that each neural network represents a function that depends on our choice of activation function for each node. In practice, we choose the same activation function for all nodes, from a small set of simple functions. It makes sense to ask just how expressive such networks can be. That is, ***can any function be reasonably approximated by a neural network?*** For a fixed function, ***what kind of archicture do we need in order to approximate it?*** Deep (multiple layers) or wide (many hidden nodes in one layer)?

![](figs/activation-functions.png)

Let's get to work.  First, we import our basic libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# `Keras` Basics 

## Keras Installation

If you haven't already, please install `Keras` using the instructions found at [https://keras.io/#installation](https://keras.io/#installation)

Choose the TensorFlow installation instructions (found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/) ).

Note the following:

* cuDNN is only required if your machine has an NVidia graphics card (GPU)
* For this tutorial, HDF5 and h5py are not required
* graphviz and pydot are not required

In [ ]:
import keras
keras.__version__

# Exploring Neural Networks
Let's try to redo the problem from last week.  Recall that we had a function $$f\left(x\right) = \exp\left(-x^{2}\right)$$ and we wanted to use a neural network to approximate that function.  This week, we will use `keras` to do the true optimization.

First, we import the necessary `keras` modules.

In [ ]:
from keras import models
from keras import layers

Now we set up a network.  Let's do a single hidden layer with two neurons.  Last week we tried to manually tune the weights and things didn't work so well.

Before we get started, we need to create some data.  We will generate data points from an underlying function (here the Guassian).  Then we will use the `sklearn` `train_test_split` method to split the dataset into training and testing portions.  Remember that we train a machine learning algorithm on the training set and then assess the algorithm's performance on the test set.

In [ ]:
from sklearn.model_selection import train_test_split

n_samples = 1000 # set the number of samples to take for each toy dataset
test_size = 0.3 # set the proportion of toy data to hold out for testing
random_seed = 1 # set the random seed to make the experiment reproducible 
np.random.seed(random_seed)

# define a function
f = lambda x: np.exp(-x * x)
X = np.random.permutation(np.linspace(-5, 5, n_samples)) # choose some points from the function - this is our toy dataset 
Y = f(X)

# create training and testing data from this set of points
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)

Let's create a neural network model with `keras`.  We're going to use a single layer and just $2$ neurons in that layer.  To be consisten with last week, we will start with the sigmoid activation function.  We also choose a linear output layer like last time.  The loss function is selected to be the mean squared error.  In addition to these choices we must also specify our initial weights as well as the optimization method that will be used to minimize the loss function.

In [ ]:
H = 2 # number of nodes in the layer
input_dim = 1 # input dimension: just x

model = models.Sequential() # create sequential multi-layer perceptron

# layer 0, our hidden layer
model.add(layers.Dense(H, input_dim=input_dim, 
                kernel_initializer='normal', 
                activation='sigmoid'))
# layer 1
model.add(layers.Dense(1, kernel_initializer='normal', 
                activation='linear')) 

# configure the model
model.compile(loss='mean_squared_error', optimizer='adam')

Our model is now ready to use.  We haven't trained it yet, but we'll do that now using the `fit` method.  Notice that we also need to specify the *batch size* for the stochastic gradient decent algorithm as well as the number of epochs to run.

In [ ]:
# fit the model
model_history = model.fit(X_train, Y_train, batch_size=128, epochs=500, verbose=1)

Great!  We've trained a model.  Now it's time to explore the results.  The first thing to notice is the value of the loss function.  Does it look okay to you?

Let's see what the results look like.

In [ ]:
# use our model to predict in the range we want
X_range = np.linspace(-5, 5, 1000)
y_pred = model.predict(X_range)

fig, ax = plt.subplots(1, 1, figsize=(14,8))
ax.scatter(X_train, Y_train, label='Training data')
ax.plot(X_range, y_pred, lw=4, color='r', label='MLP with one hidden layer')
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_title('Toy data set for regression', fontsize=24)
ax.tick_params(labelsize=20)

ax.legend(loc=1, fontsize=20)

plt.show()

How do things look?

### Exercise
Explore the following:
* Different activation functions
  - ReLU
  - $tanh$
  - [Something else?](https://keras.io/activations/)
* Change the number of neurons
  - Increase it from $2$ to $4$ at first
  - What number seems to work well?
* Try to play with the number of epochs and the batch size.  Do these have an impact?

### Exercise
Plot the loss function as a function of the epochs.

#### Hint
You can access the loss function values with the command:
```python
model_history.history['loss']
```

#### Model Performance
How good is the model?  We can compute the $R^{2}$ score to get a sense of the model performance.

In [ ]:
# evaluate the training and testing performance of your model 
# note: you should extract and check both the loss function and your evaluation metric
from sklearn.metrics import r2_score as r2

train_score = model.evaluate(X_train, Y_train, verbose=1)
print('Train loss:', train_score)
print('Train R2:', r2(Y_train, model.predict(X_train)))

test_score = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', test_score)
print('Test R2:', r2(Y_test, model.predict(X_test)))

### Exercise
Plot the train/test performace against the number of hidden nodes, H.

**Hint**
You should loop over enough neurons to make a decent plot.  However, if you make $H$ too big then it will take a long time to get a solution.  Choosing $H = [2, 4, 6, 8, 10]$ may be a good place to start.  You may want to try a larger list at some point in your free time.

## Going Deeper:  Changing the Number of Layers

Now fix a width $H$ and let's fit a MLP network with **multiple** hidden layers, each with the same width. Start with sigmoid or $tanh$ activation functions for the hidden nodes and linear activation for the output. 

***Experiment with the number of layers and observe the effect of this on the quality of the fit.***  You want to think about issues like computational effeciency and generalizability of this type of modeling. You should compare the MLP to your model with a single hidden layer (in terms of quality of fit, efficiency and generalizability).

In [ ]:
# number of hidden nodes
H = 4
# input dimension
input_dim = 1

# create sequential multi-layer perceptron
model2 = Sequential()
# layer 0
# model2.add(...) 

# layer 1
# model2.add(...) 

# layer 2
# model2.add(...) 

# layer 3
# model2.add(...) 

# layer 4
# model2.add(...) 

# layer 5
# model2.add(...) 

# layer 6
# model2.add(...) 

# configure the model
# model2.compile(...)

In [ ]:
# fit the model
# model2.fit(...)

In [ ]:
# use our model to predict in the range we want
X_range = np.linspace(-4, 4, 500)
y_pred = model2.predict(X_range)

fig, ax = plt.subplots(1,1, figsize=(10,6))
ax.scatter(X_train, Y_train, label='Training data')
ax.plot(X_range, y_pred, lw=4, color='r', label='MLP with 6 hidden layers')

ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_title('Toy data set for regression', fontsize=24)
ax.tick_params(labelsize=20)

In [ ]:
# evaluate the training and testing performance of your model 
# note: you should extract check both the loss function and your evaluation metric
score = model2.evaluate(X_train, Y_train, verbose=0)
print('Train loss:', score)
print('Train R2:', r2(Y_train, model2.predict(X_train)))

In [ ]:
score = model2.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score)
print('Test R2:', r2(Y_test, model2.predict(X_test)))

### Exercise
What if we wanted to approximate a different function $f$ with MLP's?  Consider the functions $f_{1}\left(x\right) = x\sin\left(x\right)$ and $f_{2}\left(x\right) = \left|x\right|$.  Can you create a network that can represent these functions?

***Experiment with approximating a few different non-linear functions with wide but shallow networks as well as deep but narrow networks.***

How expressive do you think MLP's are?

# Automatic Differentiation Demo
Lecture introduced automatic differentiation.  Here we will show a *very basic* implementation for the forward mode.

Suppose we want to compute the derivative of $f = x^{3}$.  This is a polynomial (actually a monomial) function and polynomials are *elementary* (or elemental) functions.

We'd like to evaluate $f^{\prime}$ at the point $a = 3$.  We can do this by hand in this simple example:
\begin{align}
  &f\left(3\right) = 27 \\
  &f^{\prime}\left(3\right) = 3 \left(3^{2}\right) = 27
\end{align}

Let's create a very simple `Python` datastructure to handle this case.

A list comes to mind.  The first index of the list will be the function value and the second index will be the derivative value.

In [ ]:
f = [0, 0] # Initialize the list
a = 3.0 # The point we want to evaluate things at
f[0] = a**3.0
f[1] = 3.0 * a**2.0
print(f)

That worked great!  The basic idea was that we can create a library of elementary functions and their derivatives.  Whenever `python` sees that elementary function, it automatically calculates the derivative as well.  This works very well in the forward mode.  Here's a slightly more advanced implementation.

In [ ]:
class ForwardAD:
    def __init__(self, val, der=1.0):
        self.val = val
        self.der = der
    
    def __add__(self, other):
        try:
            return ForwardAD(self.val + other.val, self.der + other.der)
        except:
            return ForwardAD(self.val + other, self.der)
    
    def __radd__(self, other):
        return ForwardAD(self.val + other, self.der)
    
    def __mul__(self, other):
        try:
            return ForwardAD(self.val * other.val, self.val * other.der + self.der * other.val)
        except:
            return ForwardAD(self.val * other, self.der * other)
    
    def __rmul__(self, other):
        return ForwardAD(self.val * other, self.der * other)
    
    def __pow__(self, other):
        try:
            fa = self.val**other.val
            fpa = (other.val / self.val * self.der + np.log(self.val) * other.der) * fa
            return ForwardAD(fa, fpa)
        except:
            fa = self.val**other
            fpa = other * self.val**(other - 1.0) * self.der
            return ForwardAD(fa, fpa)
    
    def __rpow__(self, other):
        fa = self.val**other
        fpa = fa * np.log(other) * self.der
        return ForwardAD(fa, fpa)        

In [ ]:
x = ForwardAD(3.0)
y = x**3.0 + 2.0 * x**2.0 + 3.0
print(y.val, y.der)